In [37]:
import rdflib
import html
g = rdflib.Graph().parse('./fb15k-wikitop2021-yago-facts.nt', format='ntriples')
g += rdflib.Graph().parse('./fb15k-wikitop2021-yago-full-types.nt', format='ntriples')

from rdflib.namespace import RDF
for line in open('fb15k-wikitop2021.tsv').readlines()[1:]:
    subject = rdflib.URIRef( line.split('\t')[6][1:-1] ) # take the URI in column 6
    g.add( (subject, RDF.type, rdflib.URIRef('http://example.com/popularEntity')) )

In [38]:
r = g.query("""
select (count(*) as ?count) ?p ?o  where 
{
    ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .
    ?s ?p ?o .
} 
group by ?p ?o 
order by desc(?count)
limit 10
""")
for b in r.bindings:
    print(*[f'{x.split("/")[-1][:40]:40s}' for x in b.values()])


22-rdf-syntax-ns#type                    popularEntity                            186                                     
22-rdf-syntax-ns#type                    Human                                    134                                     
knowsLanguage                            English_language                         118                                     
nationality                              United_States                            93                                      
hasOccupation                            Actor                                    77                                      
hasOccupation                            Film_producer                            51                                      
hasOccupation                            Screenwriter                             37                                      
hasOccupation                            Film_director                            32                                      
memberOf        

In [53]:
%load_ext ipython_sparql_pandas

The ipython_sparql_pandas extension is already loaded. To reload it, use:
  %reload_ext ipython_sparql_pandas


In [40]:
%%sparql http://DESKTOP-D6TQ7G4:7200/repositories/20qs -s res 
select (count(*) as ?count) ?p ?o  where 
{
    ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .
    ?s ?p ?o .
} 
group by ?p ?o 
order by desc(?count)
limit 10

,count,p,o
0,186,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://example.com/popularEntity
1,134,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://yago-knowledge.org/resource/Human
2,118,http://schema.org/knowsLanguage,http://yago-knowledge.org/resource/English_lan...
3,93,http://schema.org/nationality,http://yago-knowledge.org/resource/United_States
4,77,http://schema.org/hasOccupation,http://yago-knowledge.org/resource/Actor
5,51,http://schema.org/hasOccupation,http://yago-knowledge.org/resource/Film_producer
6,37,http://schema.org/hasOccupation,http://yago-knowledge.org/resource/Screenwriter
7,32,http://schema.org/hasOccupation,http://yago-knowledge.org/resource/Film_director
8,29,http://schema.org/memberOf,http://yago-knowledge.org/resource/Democratic_...
9,25,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://schema.org/Country


In [41]:
count=res['count']
count

0    186
1    134
2    118
3     93
4     77
5     51
6     37
7     32
8     29
9     25
Name: count, dtype: int64

In [65]:
#from the query results extract a list that contains the counts after splitting on certain attributes
PosAttr=['?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .']
NegAttr=[]
def extractCountAndAttr(result):
    listOfSplits = res['count']
    listOfObj = res['o']
    listOfPred= res['p']
    return(listOfSplits,listOfObj,listOfPred)

#function to approximate index element of list that provides best split
def bestSplit(listOfNumbers, ElementsLeft):
    currentBest=1
    indexBest=-1
    for i in range(len(listOfNumbers)):
        splitScore = abs((listOfNumbers[i]/ElementsLeft)-0.5)
        if splitScore<currentBest:
            currentBest=splitScore
            indexBest=i
    return indexBest


def generateQuestion(listOfPred, listOfObj, index):
    userAnswer = input(f'Does the thing you are looking for have the attribute: {listOfPred[index].split("/")[-1]} {listOfObj[index].split("/")[-1]} ?' )
    if userAnswer.startswith('y'):
        PosAttr.append('?s <' + str(listOfPred[index]) +'> <'+str(listOfObj[index])+ '>.' )   
    elif userAnswer.startswith('n'):
        NegAttr.append('FILTER NOT EXISTS {{ \n ?s <' + str(listOfPred[index]) +'> <'+str(listOfObj[index])+ '>. }}' )   
    else:
        generateQuestion(listOfPred, listOfObj, index+1)
    NegFilters = "\n".join(NegAttr)
    PosFilters = "\n".join(PosAttr)
    return (PosFilters, NegFilters)


  
def updateQuery( PosFilters, NegFilters):

    FilterQuestion =  f"""
    select (count(*) as ?count) ?p ?o  where 
    {{

    {PosFilters}
     ?s ?p ?o .
    {NegFilters}
    }} 
    group by ?p ?o 
    order by desc(?count)
    limit 25
    """
        
    return FilterQuestion




In [ ]:
qres = g.query("""
select (count(*) as ?count) ?p ?o  where 
{
    ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .
    ?s ?p ?o .
} 
group by ?p ?o 
order by desc(?count)
limit 10
""")
for b in qres.bindings:
    print(*[f'{x.split("/")[-1][:40]:40s}' for x in b.values()])
availableSplits, listOfObj ,listOfPred = extractCountAndAttr(r)
index= bestSplit(availableSplits, availableSplits[0])


22-rdf-syntax-ns#type                    popularEntity                            186                                     
22-rdf-syntax-ns#type                    Human                                    134                                     
knowsLanguage                            English_language                         118                                     
nationality                              United_States                            93                                      
hasOccupation                            Actor                                    77                                      
hasOccupation                            Film_producer                            51                                      
hasOccupation                            Screenwriter                             37                                      
hasOccupation                            Film_director                            32                                      
memberOf        

In [16]:
#without printing results
i=0
while i< 20:
   
        
    availableSplits, listOfObj ,listOfPred = extractCountAndAttr(qres)
    if availableSplits[0]<=1:
        break
    index= bestSplit(availableSplits, availableSplits[0])
    PosFilters, NegFilters = generateQuestion(listOfPred, listOfObj, index)
    query = updateQuery(PosFilters, NegFilters)
    qres = g.query(query)

finalResult= f"""
            select distinct ?s
            where 
            {{
            
            {PosFilters}
            ?s ?p ?o
            {NegFilters}
            }} """
finalRes = g.query(finalResult)
for b in finalRes.bindings:
    print(*[f'Your Answer is: {x.split("/")[-1]}' for x in b.values()][0:10]) 

Does the thing you are looking for have the attribute: nationality United_States ? n
Does the thing you are looking for have the attribute: 22-rdf-syntax-ns#type Human ? y
Does the thing you are looking for have the attribute: nationality United_Kingdom ? l
Does the thing you are looking for have the attribute: hasOccupation Actor ? l
Does the thing you are looking for have the attribute: hasOccupation Singer-songwriter ? y
Does the thing you are looking for have the attribute: award Rock_and_Roll_Hall_of_Fame ? y
Does the thing you are looking for have the attribute: award Grammy_Lifetime_Achievement_Award ? y
Does the thing you are looking for have the attribute: birthPlace Liverpool ? y
Does the thing you are looking for have the attribute: alumniOf Calderstones_School ? y


Your Answer is: John_Lennon


In [13]:
#Always pritning the posible spilits 
i=0
while i< 20:
    if availableSplits[0]<=1:
        break
        
    availableSplits, listOfObj ,listOfPred = extractCountAndAttr(qres)
    index= bestSplit(availableSplits, availableSplits[0])
    PosFilters, NegFilters = generateQuestion(listOfPred, listOfObj, index)
    query = updateQuery(PosFilters, NegFilters)
    qres = g.query(query)
    for b in qres.bindings:
        print(*[f'{x.split("/")[-1][:40]:40s}' for x in b.values()][0:10]) 

finalResult= f"""
            select distinct ?s
            where 
            {{
            
            {PosFilters}
            ?s ?p ?o
            {NegFilters}
            }} """
finalRes = g.query(finalResult)
for b in finalRes.bindings:
    print(*[f'Your Answer is: {x.split("/")[-1]}' for x in b.values()][0:10]) 

Does the thing you are looking for have the attribute: nationality United_States ? n


22-rdf-syntax-ns#type                    popularEntity                            93                                      
22-rdf-syntax-ns#type                    Human                                    41                                      
knowsLanguage                            English_language                         34                                      
22-rdf-syntax-ns#type                    Country                                  25                                      
memberOf                                 United_Nations                           21                                      
nationality                              United_Kingdom                           20                                      
22-rdf-syntax-ns#type                    Sovereign_state                          20                                      
memberOf                                 International_Bank_for_Reconstruction_an 19                                      
memberOf        

Does the thing you are looking for have the attribute: 22-rdf-syntax-ns#type Human ? n


22-rdf-syntax-ns#type                    popularEntity                            52                                      
22-rdf-syntax-ns#type                    Country                                  25                                      
memberOf                                 United_Nations                           21                                      
22-rdf-syntax-ns#type                    Sovereign_state                          20                                      
memberOf                                 International_Bank_for_Reconstruction_an 19                                      
memberOf                                 Multilateral_Investment_Guarantee_Agency 19                                      
memberOf                                 International_Telecommunication_Union    19                                      
memberOf                                 International_Finance_Corporation        19                                      
memberOf        

Does the thing you are looking for have the attribute: 22-rdf-syntax-ns#type Country ? n


...

22-rdf-syntax-ns#type                    City                                     2                                       
22-rdf-syntax-ns#type                    popularEntity                            2                                       
image                                    EmpireStateNewYokCity.jpg                1                                       
containsPlace                            Staten_Island                            1                                       
geo                                      geo:40.67000000000033,-73.9400000000006  1                                       
url                                                                               1                                       
containedInPlace                         New_York_metropolitan_area               1                                       
containedInPlace                         New_York_(state)                         1                                       
iataCode        

Does the thing you are looking for have the attribute: image EmpireStateNewYokCity.jpg ? y


image                                    EmpireStateNewYokCity.jpg                1                                       
containsPlace                            Staten_Island                            1                                       
geo                                      geo:40.67000000000033,-73.9400000000006  1                                       
url                                                                               1                                       
containedInPlace                         New_York_metropolitan_area               1                                       
containedInPlace                         New_York_(state)                         1                                       
iataCode                                 NYC                                      1                                       
location                                 New_York_metropolitan_area               1                                       
location        

Does the thing you are looking for have the attribute: image EmpireStateNewYokCity.jpg ? y


image                                    EmpireStateNewYokCity.jpg                1                                       
containsPlace                            Staten_Island                            1                                       
geo                                      geo:40.67000000000033,-73.9400000000006  1                                       
url                                                                               1                                       
containedInPlace                         New_York_metropolitan_area               1                                       
containedInPlace                         New_York_(state)                         1                                       
iataCode                                 NYC                                      1                                       
location                                 New_York_metropolitan_area               1                                       
location        

In [50]:
#Cell To Reset the program:
PosAttr=['?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .']
NegAttr=[]
qres = g.query("""
select (count(*) as ?count) ?p ?o  where 
{
    ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .
    ?s ?p ?o .
} 
group by ?p ?o 
order by desc(?count)
limit 10
""")
availableSplits, listOfObj ,listOfPred = extractCountAndAttr(qres)
index= bestSplit(availableSplits, availableSplits[0])

In [74]:
availableSplits, listOfObj ,listOfPred = extractCountAndAttr(qres)
index= bestSplit(availableSplits, availableSplits[0])
PosFilters, NegFilters = generateQuestion(listOfPred, listOfObj, index)
query = updateQuery(PosFilters, NegFilters)



Does the thing you are looking for have the attribute: nationality United_States ? y


In [123]:
from SPARQLWrapper import SPARQLWrapper , JSON, CSV, TSV, TURTLE
import pandas as pd

In [124]:
sparql = SPARQLWrapper("http://DESKTOP-D6TQ7G4:7200/repositories/20qs")

In [161]:
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
qres = sparql.query().convert()

    


In [159]:

for result in qres['head']['results']:
    print(result)

KeyError: 'results'

In [164]:
for i,j in qres.items():
    print(i,j)

head {'vars': ['count', 'p', 'o']}
results {'bindings': [{'count': {'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'type': 'literal', 'value': '93'}, 'p': {'type': 'uri', 'value': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'}, 'o': {'type': 'uri', 'value': 'http://example.com/popularEntity'}}, {'count': {'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'type': 'literal', 'value': '93'}, 'p': {'type': 'uri', 'value': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'}, 'o': {'type': 'uri', 'value': 'http://yago-knowledge.org/resource/Human'}}, {'count': {'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'type': 'literal', 'value': '93'}, 'p': {'type': 'uri', 'value': 'http://schema.org/nationality'}, 'o': {'type': 'uri', 'value': 'http://yago-knowledge.org/resource/United_States'}}, {'count': {'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'type': 'literal', 'value': '84'}, 'p': {'type': 'uri', 'value': 'http://schema.org/knowsLanguage'}, 'o': {'type':

,head,results
vars,"[count, p, o]",NaN
bindings,NaN,[{'count': {'datatype': 'http://www.w3.org/200...
